In [ ]:
import csv
import numpy as np

In [ ]:
#Import refined labels

#expert consensus labels
with open('labels1.csv', 'r') as f:
    reader = csv.reader(f)
    labels1 = [row for row in reader]
#crowdsourced consensus labels
with open('labels2.csv', 'r') as f:
    reader = csv.reader(f)
    labels2 = [row for row in reader]

In [ ]:
#Confusion matrix -- expert and crowd

#Function to calculate IOU of bounding boxes
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x_overlap = max(0, min(x1+w1, x2+w2) - max(x1, x2))
    y_overlap = max(0, min(y1+h1, y2+h2) - max(y1, y2))
    intersection = x_overlap * y_overlap
    union = (w1 * h1) + (w2 * h2) - intersection
    return intersection / union

#Create a dictionary to store the bounding boxes and class IDs for each image
boxes1 = {}
classes1 = {}
boxes2 = {}
classes2 = {}
for row in labels1:
    if row[1] not in boxes1:
        boxes1[row[1]] = []
        classes1[row[1]] = []
    boxes1[row[1]].append([float(x) for x in row[2:6]])
    classes1[row[1]].append(row[6])

for row in labels2:
    if row[1] not in boxes2:
        boxes2[row[1]] = []
        classes2[row[1]] = []
    boxes2[row[1]].append([float(x) for x in row[2:6]])
    classes2[row[1]].append(row[6])

#Compare the two sets of labels to identify the matching bounding boxes
conf_matrix = np.zeros((len(set(sum(classes1.values(), []))), len(set(sum(classes2.values(), [])))))
class_map1 = {c: i for i, c in enumerate(sorted(set(sum(classes1.values(), []))))}
class_map2 = {c: i for i, c in enumerate(sorted(set(sum(classes2.values(), [])))))}
for i, (img_name1, img_boxes1) in enumerate(boxes1.items()):
    for j, (img_name2, img_boxes2) in enumerate(boxes2.items()):
        for box1, c1 in zip(img_boxes1, classes1[img_name1]):
            for box2, c2 in zip(img_boxes2, classes2[img_name2]):
                if c1 == c2:
                    iou = calculate_iou(box1, box2)
                    if iou >= 0.5:
                        conf_matrix[class_map1[c1], class_map2[c2]] += 1

#Create a confusion matrix to visualize the results
print(conf_matrix)

In [ ]:
#Visualize results
import matplotlib.pyplot as plt
import seaborn as sns

# Step 5: Visualize the confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', ax=ax)

# Add labels and title
ax.set_xlabel('Labels2')
ax.set_ylabel('Labels1')
ax.set_title('Confusion Matrix')

# Add class names as tick labels
ax.set_xticklabels(list(class_map2.keys()))
ax.set_yticklabels(list(class_map1.keys()))

plt.show()